In [1]:
import numpy as np
import pandas as pd


Base data (film to book linkage)

In [2]:
films_to_books = pd.read_parquet("./data/films_to_books.parquet")
films_to_books.head()

,Fiction work(s),Film adaptation(s),author,book_published_date,film_published_date,book_title,film_title
0,"The 25th Hour (2001), David Benioff",25th Hour (2002),DAVID BENIOFF,2001,2002,THE 25TH HOUR,25TH HOUR
1,"3 Assassins (グラスホッパー, Gurasuhoppā) (2004), Kōt...",Grasshopper (2015),KŌTARŌ ISAKA,2004,2015,3 ASSASSINS,GRASSHOPPER
2,"4.50 from Paddington (1957), Agatha Christie","Murder, She Said (1961)",AGATHA CHRISTIE,1957,1961,4.50 FROM PADDINGTON,"MURDER, SHE SAID"
3,"4.50 from Paddington (1957), Agatha Christie",Crime Is Our Business (French: Le Crime est no...,AGATHA CHRISTIE,1957,2008,4.50 FROM PADDINGTON,CRIME IS OUR BUSINESS
4,"58 Minutes (1987), Walter Wager",Die Hard 2 (1990),WALTER WAGER,1987,1990,58 MINUTES,DIE HARD 2


In [3]:
films_to_books["book_published_date"].unique()

array(['2001', '2004', '1957', '1987', '1936', '1998', '1996', '1985',
       '1970', '1969', '2000', '1971', '1999', '1816', '1911', '1966',
       '1996–present', '1866', '1824', '1884', '1883', None, '1876',
       '1959', '1923', '1935', '2017', '1955', '1979', '1953', '1921',
       '1961', '1968', '1887', '1960', '1946', '1919', '1929', '1992',
       '2009', '1895', '1976', '1993', '1978', '1948', '1990', '1965',
       '1995', '1991', '1976–2008', '1925', '1927', '1943', '1977',
       '2005', '1958', '1942', '1939', '1957–2012', '1956', '2014',
       '1944', '1909', '1908', '1937', '1933', '1938', '1951', '1873',
       '1949', '1963', '1983', '1982', '1877', '1854', '1912', '1975',
       '1914', '1997', '1981', '1989', '1931', '1928', '1954', '1972',
       '1900', '1947', '1890', '2003', '1907', '1924', '1926', '1945',
       '1930', '1932', '1885', '1880', '1941', '1952', '1932–1968',
       '1888', '1941–1989', '1984', '1964', '1973', '1916', '1934',
       '1906', '1980

Delete any publication date that isn't a number

In [4]:
films_to_books.loc[
    films_to_books["book_published_date"].str.isnumeric() == False,
    "book_published_date",
] = np.nan

films_to_books["book_published_date"] = films_to_books["book_published_date"].astype(
    float
)

## Join book ratings data

In [5]:
book_ratings = pd.read_parquet("./data/book_ratings.parquet")
print(book_ratings.shape)
book_ratings.head()

(10000, 8)


,goodreads_book_id,authors,original_publication_year,original_title,title,average_rating,small_image_url,average_rating_normalised
0,2767052,SUZANNE COLLINS,2008.0,THE HUNGER GAMES,"THE HUNGER GAMES (THE HUNGER GAMES, #1)",4.34,https://images.gr-assets.com/books/1447303603s...,86.8
1,3,"J.K. ROWLING, MARY GRANDPRÉ",1997.0,HARRY POTTER AND THE PHILOSOPHER'S STONE,HARRY POTTER AND THE SORCERER'S STONE (HARRY P...,4.44,https://images.gr-assets.com/books/1474154022s...,88.8
2,41865,STEPHENIE MEYER,2005.0,TWILIGHT,"TWILIGHT (TWILIGHT, #1)",3.57,https://images.gr-assets.com/books/1361039443s...,71.4
3,2657,HARPER LEE,1960.0,TO KILL A MOCKINGBIRD,TO KILL A MOCKINGBIRD,4.25,https://images.gr-assets.com/books/1361975680s...,85.0
4,4671,F. SCOTT FITZGERALD,1925.0,THE GREAT GATSBY,THE GREAT GATSBY,3.89,https://images.gr-assets.com/books/1490528560s...,77.8


Let's try joining:

- only on title
- title AND publication date
- title AND author (and date?)
- title (fuzzy match)

In [6]:
print(len(films_to_books))

merged_book_ratings = films_to_books.merge(
    book_ratings, left_on="book_title", right_on="original_title", how="inner"
)

print(
    f"{len(merged_book_ratings)} books merged on title alone ({100 * (len(merged_book_ratings) / len(films_to_books)):.1f}%)"
)

7320
2270 books merged on title alone (31.0%)


Title AND date

In [7]:
print(len(films_to_books))

merged_book_ratings_2 = films_to_books.merge(
    book_ratings,
    left_on=["book_title", "book_published_date"],
    right_on=["original_title", "original_publication_year"],
    how="inner",
)

print(
    f"{len(merged_book_ratings_2)} books merged on title and date ({100 * (len(merged_book_ratings_2) / len(films_to_books)):.1f}%)"
)

7320
1880 books merged on title and date (25.7%)


Title and author (not date)

In [8]:
print(len(films_to_books))

merged_book_ratings_3 = films_to_books.merge(
    book_ratings,
    left_on=["book_title", "author"],
    right_on=["original_title", "authors"],
    how="inner",
)

print(
    f"{len(merged_book_ratings_3)} books merged on title and author ({100 * (len(merged_book_ratings_3) / len(films_to_books)):.1f}%)"
)

7320
1420 books merged on title and author (19.4%)


### Fuzzy string matching

- for each book in the film-to-book link data, find the closest title in the book ratings data by string similarity (Levenshtein distance)
    - include a minimum similarity threshold!
- join the book ratings on this closest match

In [9]:
from Levenshtein import distance

Define a function to get similarity

In [26]:
def most_similar_title(title, all_titles, max_distance=5):
    distances = [
        (s, distance(title, s))
        for s in all_titles
        if distance(title, s) <= max_distance
    ]

    if len(distances) == 0:
        return np.nan

    closest_match = min(distances, key=lambda x: x[1])[0]

    return closest_match


all_book_titles = book_ratings["original_title"].dropna().tolist()

most_similar_title("afughwerydfghdsggd", all_book_titles)

nan

Call this function on all book titles in book-to-film mapping and see results

In [29]:
all_book_titles = book_ratings["original_title"].dropna().tolist()

films_to_books["closest_book_match"] = films_to_books["book_title"].apply(
    lambda x: most_similar_title(x, all_book_titles, max_distance=3)
)

films_to_books.head()

,Fiction work(s),Film adaptation(s),author,book_published_date,film_published_date,book_title,film_title,closest_book_match
0,"The 25th Hour (2001), David Benioff",25th Hour (2002),DAVID BENIOFF,2001.0,2002,THE 25TH HOUR,25TH HOUR,NaN
1,"3 Assassins (グラスホッパー, Gurasuhoppā) (2004), Kōt...",Grasshopper (2015),KŌTARŌ ISAKA,2004.0,2015,3 ASSASSINS,GRASSHOPPER,NaN
2,"4.50 from Paddington (1957), Agatha Christie","Murder, She Said (1961)",AGATHA CHRISTIE,1957.0,1961,4.50 FROM PADDINGTON,"MURDER, SHE SAID",4.50 FROM PADDINGTON
3,"4.50 from Paddington (1957), Agatha Christie",Crime Is Our Business (French: Le Crime est no...,AGATHA CHRISTIE,1957.0,2008,4.50 FROM PADDINGTON,CRIME IS OUR BUSINESS,4.50 FROM PADDINGTON
4,"58 Minutes (1987), Walter Wager",Die Hard 2 (1990),WALTER WAGER,1987.0,1990,58 MINUTES,DIE HARD 2,NaN


In [ ]:
films_to_books[
    (films_to_books["book_title"] != films_to_books["closest_book_match"])
    & (films_to_books["closest_book_match"].notna())
]

,Fiction work(s),Film adaptation(s),author,book_published_date,film_published_date,book_title,film_title,closest_book_match
5,"69 (シクスティナイン, Shikusutinain) (1987), Ryu Murakami",69 (2004),RYU MURAKAMI,1987.0,2004,69,69,IT
6,"The A.B.C. Murders (1936), Agatha Christie",The Alphabet Murders (1966),AGATHA CHRISTIE,1936.0,1966,THE A.B.C. MURDERS,THE ALPHABET MURDERS,THE ABC MURDERS
91,"Aelita (1923) (a. k. a. Aelita or, The Decline...",Aelita (1924),ALEXEI TOLSTOY,1923.0,1924,AELITA,AELITA,LOLITA
95,"After Hours (1979), Edwin Torres",Carlito's Way (1993),EDWIN TORRES,1979.0,1993,AFTER HOURS,CARLITO'S WAY,AFTER YOU
117,"All the King's Men (1946), Robert Penn Warren",All the King's Men (1949),ROBERT PENN WARREN,1946.0,1949,ALL THE KING'S MEN,ALL THE KING'S MEN,ALL THE KING’S MEN
...,...,...,...,...,...,...,...,...
7239,"Cocoon (1985), David Saperstein",Cocoon: The Return (1988),DAVID SAPERSTEIN,1985.0,1988,COCOON,COCOON: THE RETURN,CONGO
7260,"Conagher (1968), Louis L'Amour",Conagher (1991)[N 1],LOUIS L'AMOUR,1968.0,1991,CONAGHER,CONAGHER,VOYAGER
7308,"Creator (1980), Jeremy Leven",Creator (1985),JEREMY LEVEN,1980.0,1985,CREATOR,CREATOR,PREDATOR
7317,"Cyborg (1972), Martin Caidin",The Six Million Dollar Man (1973)[N 1],MARTIN CAIDIN,1972.0,1973,CYBORG,THE SIX MILLION DOLLAR MAN,ICEBERG


This is terrible - too many false positive even with an allowable string distance of 3 or fewer characters...

### Conclusion

Join books on exact title only

In [34]:
merged_book_ratings = films_to_books.merge(
    book_ratings, left_on="book_title", right_on="original_title", how="inner"
)

merged_book_ratings.head()

,Fiction work(s),Film adaptation(s),author,book_published_date,film_published_date,book_title,film_title,closest_book_match,goodreads_book_id,authors,original_publication_year,original_title,title,average_rating,small_image_url,average_rating_normalised
0,"4.50 from Paddington (1957), Agatha Christie","Murder, She Said (1961)",AGATHA CHRISTIE,1957.0,1961,4.50 FROM PADDINGTON,"MURDER, SHE SAID",4.50 FROM PADDINGTON,140278,AGATHA CHRISTIE,1957.0,4.50 FROM PADDINGTON,"4:50 FROM PADDINGTON (MISS MARPLE, #8)",3.92,https://s.gr-assets.com/assets/nophoto/book/50...,78.4
1,"4.50 from Paddington (1957), Agatha Christie",Crime Is Our Business (French: Le Crime est no...,AGATHA CHRISTIE,1957.0,2008,4.50 FROM PADDINGTON,CRIME IS OUR BUSINESS,4.50 FROM PADDINGTON,140278,AGATHA CHRISTIE,1957.0,4.50 FROM PADDINGTON,"4:50 FROM PADDINGTON (MISS MARPLE, #8)",3.92,https://s.gr-assets.com/assets/nophoto/book/50...,78.4
2,"About a Boy (1998), Nick Hornby",About a Boy (2002),NICK HORNBY,1998.0,2002,ABOUT A BOY,ABOUT A BOY,ABOUT A BOY,4271,NICK HORNBY,1998.0,ABOUT A BOY,ABOUT A BOY,3.79,https://images.gr-assets.com/books/1382004144s...,75.8
3,"Absolute Power (1996), David Baldacci",Absolute Power (1997),DAVID BALDACCI,1996.0,1997,ABSOLUTE POWER,ABSOLUTE POWER,ABSOLUTE POWER,15159,DAVID BALDACCI,1995.0,ABSOLUTE POWER,ABSOLUTE POWER,4.15,https://images.gr-assets.com/books/1328399707s...,83.0
4,"The Accidental Tourist (1985), Anne Tyler",The Accidental Tourist (1988),ANNE TYLER,1985.0,1988,THE ACCIDENTAL TOURIST,THE ACCIDENTAL TOURIST,THE ACCIDENTAL TOURIST,60792,"ANNE TYLER, JENNIFER BASSETT",1985.0,THE ACCIDENTAL TOURIST,THE ACCIDENTAL TOURIST,3.90,https://images.gr-assets.com/books/1502220282s...,78.0


# Films

Let's try joining:

- only on title
- title AND publication date
- title AND author (and date?)
- title (fuzzy match)

In [30]:
film_ratings = pd.read_parquet("./data/film_ratings.parquet")
print(film_ratings.shape)
film_ratings.head()

(9366, 2)


,film_title,avg_film_rating
0,#HORROR,53.400000
1,#UNFIT: THE PSYCHOLOGY OF DONALD TRUMP,77.750000
2,$ (DOLLARS),71.333333
3,$9.99,69.012821
4,'71,78.122549


In [31]:
print(len(films_to_books))

merged_film_ratings = films_to_books.merge(film_ratings, on="film_title", how="inner")

print(
    f"{len(merged_film_ratings)} films merged on title alone ({100 * (len(merged_film_ratings) / len(films_to_books)):.1f}%)"
)

7320
720 films merged on title alone (9.8%)


Fuzzy title matching for films

In [35]:
all_film_titles = film_ratings["film_title"].dropna().tolist()

films_to_books["closest_film_match"] = films_to_books["film_title"].apply(
    lambda x: most_similar_title(x, all_film_titles, max_distance=3)
)

films_to_books.head()

,Fiction work(s),Film adaptation(s),author,book_published_date,film_published_date,book_title,film_title,closest_book_match,closest_film_match
0,"The 25th Hour (2001), David Benioff",25th Hour (2002),DAVID BENIOFF,2001.0,2002,THE 25TH HOUR,25TH HOUR,NaN,NaN
1,"3 Assassins (グラスホッパー, Gurasuhoppā) (2004), Kōt...",Grasshopper (2015),KŌTARŌ ISAKA,2004.0,2015,3 ASSASSINS,GRASSHOPPER,NaN,NaN
2,"4.50 from Paddington (1957), Agatha Christie","Murder, She Said (1961)",AGATHA CHRISTIE,1957.0,1961,4.50 FROM PADDINGTON,"MURDER, SHE SAID",4.50 FROM PADDINGTON,NaN
3,"4.50 from Paddington (1957), Agatha Christie",Crime Is Our Business (French: Le Crime est no...,AGATHA CHRISTIE,1957.0,2008,4.50 FROM PADDINGTON,CRIME IS OUR BUSINESS,4.50 FROM PADDINGTON,NaN
4,"58 Minutes (1987), Walter Wager",Die Hard 2 (1990),WALTER WAGER,1987.0,1990,58 MINUTES,DIE HARD 2,NaN,NaN


In [36]:
films_to_books[films_to_books["closest_film_match"].notna()]

,Fiction work(s),Film adaptation(s),author,book_published_date,film_published_date,book_title,film_title,closest_book_match,closest_film_match
5,"69 (シクスティナイン, Shikusutinain) (1987), Ryu Murakami",69 (2004),RYU MURAKAMI,1987.0,2004,69,69,IT,1
13,"Adam Resurrected (Hebrew: אדם בן כלב) (1969), ...",Adam Resurrected (2009),YORAM KANIUK,1969.0,2009,ADAM RESURRECTED,ADAM RESURRECTED,NaN,ADAM RESURRECTED
14,"Adams Fall (2000), Sean Desmond",Abandon (2002),SEAN DESMOND,2000.0,2002,ADAMS FALL,ABANDON,NaN,ALADDIN
15,"Addie Pray (1971), Joe David Brown",Paper Moon (1973),JOE DAVID BROWN,1971.0,1973,ADDIE PRAY,PAPER MOON,NaN,PAPER MAN
17,"Adolphe (1816), Benjamin Constant",Adolphe (2002),BENJAMIN CONSTANT,1816.0,2002,ADOLPHE,ADOLPHE,NaN,ADORE
...,...,...,...,...,...,...,...,...,...
7277,"Contact (1985), Carl Sagan",Contact (1997),CARL SAGAN,1985.0,1997,CONTACT,CONTACT,CONTACT,CONTRACTED
7299,The Count of Monte Cristo (French: Le comte de...,Veta (1986),PÈRE,NaN,1986,THE COUNT OF MONTE CRISTO,VETA,NaN,GRETA
7306,"Cracking India (1991), Bapsi Sidhwa",Earth (1998),BAPSI SIDHWA,1991.0,1998,CRACKING INDIA,EARTH,NaN,EARTH
7307,"Crash (1973), J. G. Ballard",Crash (1996),J. G. BALLARD,1973.0,1996,CRASH,CRASH,CRASH,TRASH


Let's abandon fuzzy matching here too and join only on title

In [37]:
print(len(merged_book_ratings))

merged_film_ratings = merged_book_ratings.merge(
    film_ratings, on="film_title", how="inner"
)

print(merged_film_ratings.shape)

merged_film_ratings.head()

2270
(300, 17)


,Fiction work(s),Film adaptation(s),author,book_published_date,film_published_date,book_title,film_title,closest_book_match,goodreads_book_id,authors,original_publication_year,original_title,title,average_rating,small_image_url,average_rating_normalised,avg_film_rating
0,The Adventures of Sherlock Holmes (serialised ...,Sherlock Holmes (1916),ARTHUR CONAN DOYLE,NaN,1916,THE ADVENTURES OF SHERLOCK HOLMES,SHERLOCK HOLMES,THE ADVENTURES OF SHERLOCK HOLMES,3590,ARTHUR CONAN DOYLE,1892.0,THE ADVENTURES OF SHERLOCK HOLMES,THE ADVENTURES OF SHERLOCK HOLMES,4.29,https://s.gr-assets.com/assets/nophoto/book/50...,85.8,64.760000
1,The Adventures of Sherlock Holmes (serialised ...,Sherlock Holmes (1922),ARTHUR CONAN DOYLE,NaN,1922,THE ADVENTURES OF SHERLOCK HOLMES,SHERLOCK HOLMES,THE ADVENTURES OF SHERLOCK HOLMES,3590,ARTHUR CONAN DOYLE,1892.0,THE ADVENTURES OF SHERLOCK HOLMES,THE ADVENTURES OF SHERLOCK HOLMES,4.29,https://s.gr-assets.com/assets/nophoto/book/50...,85.8,64.760000
2,"The Adventures of Tom Sawyer (1876), Mark Twain",Band of Robbers (2015),MARK TWAIN,1876.0,2015,THE ADVENTURES OF TOM SAWYER,BAND OF ROBBERS,THE ADVENTURES OF TOM SAWYER,24583,"MARK TWAIN, GUY CARDWELL, JOHN SEELYE",1876.0,THE ADVENTURES OF TOM SAWYER,THE ADVENTURES OF TOM SAWYER,3.89,https://images.gr-assets.com/books/1404811979s...,77.8,68.250000
3,"After (2017), Anna Todd",After (2019),ANNA TODD,2017.0,2019,AFTER,AFTER,AFTER,22557520,ANNA TODD,2014.0,AFTER,"AFTER (AFTER, #1)",3.74,https://images.gr-assets.com/books/1438077259s...,74.8,39.652174
4,"After (2017), Anna Todd",After (2019),ANNA TODD,2017.0,2019,AFTER,AFTER,AFTER,6338619,AMY EFAW,2009.0,AFTER,AFTER,3.73,https://s.gr-assets.com/assets/nophoto/book/50...,74.6,39.652174


In [50]:
final_data = (
    merged_film_ratings[
        [
            "book_title",
            "author",
            "book_published_date",
            "average_rating_normalised",
            "film_title",
            "film_published_date",
            "avg_film_rating",
        ]
    ]
    .drop_duplicates()
    .rename(columns={"average_rating_normalised": "avg_book_rating"})
)

print(final_data.shape)
final_data.sort_values("book_title")

(30, 7)


,book_title,author,book_published_date,avg_book_rating,film_title,film_published_date,avg_film_rating
29,A CLOCKWORK ORANGE,ANTHONY BURGESS,1962.0,79.6,VINYL,1965,48.333333
3,AFTER,ANNA TODD,2017.0,74.8,AFTER,2019,39.652174
4,AFTER,ANNA TODD,2017.0,74.6,AFTER,2019,39.652174
5,ANGELS & DEMONS,DAN BROWN,2000.0,77.0,ANGELS & DEMONS,2009,54.865782
6,ANNIHILATION,JEFF VANDERMEER,2014.0,72.4,ANNIHILATION,2018,78.531963
7,BEAUTIFUL CREATURES,"KAMI GARCIA, MARGARET STOHL",2009.0,75.2,BEAUTIFUL CREATURES,2013,57.332192
8,BEL-AMI,GUY DE MAUPASSANT,1885.0,76.2,BEL AMI,1939,49.108696
9,BEL-AMI,GUY DE MAUPASSANT,1885.0,76.2,BEL AMI,2005,49.108696
10,BEL-AMI,GUY DE MAUPASSANT,1885.0,76.2,BEL AMI,2012,49.108696
11,BELOVED,TONI MORRISON,1987.0,75.4,BELOVED,1998,61.102941


Quick check - any films better?

In [51]:
final_data[final_data["avg_film_rating"] > final_data["avg_book_rating"]]

,book_title,author,book_published_date,avg_book_rating,film_title,film_published_date,avg_film_rating
6,ANNIHILATION,JEFF VANDERMEER,2014.0,72.4,ANNIHILATION,2018,78.531963
19,CALL ME BY YOUR NAME,ANDRÉ ACIMAN,2007.0,82.6,CALL ME BY YOUR NAME,2017,87.500000
25,CHRISTINE,STEPHEN KING,1983.0,74.4,CHRISTINE,1983,74.535294


Some results but clearly lacking in film review data. Need to find another source of film review data that includes release date so we can join on both and try again.